In [ ]:
import os
import numpy as np
import torch as pt
import pandas as pd
from glob import glob
from tqdm import tqdm
from time import time

import esm.inverse_folding

In [ ]:
# parameters
device = pt.device("cuda")

# load model
model, alphabet = esm.pretrained.esm_if1_gvp4_t16_142M_UR50()
model = model.eval().to(device)

In [ ]:
# locate benchark structures
pdb_filepaths = glob("tmp/*.pdb")

profiling = []
for pdb_filepath in tqdm(pdb_filepaths):
    try:
        t0 = time()
        # load structure
        structure = esm.inverse_folding.util.load_structure(pdb_filepath)
        coords, seq = esm.inverse_folding.util.extract_coords_from_structure(structure)
    
        # sample sequence
        sampled_seq = model.sample(coords, temperature=1e-6, device=device)
    
        # profiling
        t1 = time()
        sid = os.path.basename(pdb_filepath).split('.')[0]
        profiling.append({'sid': sid, 'dt': t1-t0})
    except Exception as e:
        print(e)

# save profiling
dfp = pd.DataFrame(profiling)
dfp.to_csv("results/profiling_esm_ext.csv", index=False)